In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("./BS_Model_Dataset.csv" )
df.head()

/var/folders/bn/0_n9dq9s6m51l0sffgs_yvtw0000gn/T/ipykernel_21004/1691883355.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./BS_Model_Dataset.csv" )


,Time,Price,Volume,Type,Stock_Price_S,Strike_Price_K,Time_to_Maturity_T,Risk_Free_Rate_r,Volatility_sigma
0,9:15:00,8.29,100.0,2B,8.29,8.7045,0.25,0.03,0.25
1,9:15:00,8.31,100.0,2B,8.31,8.7255,0.25,0.03,0.25
2,9:15:00,8.38,100.0,2B,8.38,8.7990,0.25,0.03,0.25
3,9:15:00,8.29,100.0,2B,8.29,8.7045,0.25,0.03,0.25
4,9:15:00,8.35,100.0,2B,8.35,8.7675,0.25,0.03,0.25


In [3]:
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S') 
df['Time'] = df['Time'].dt.strftime('%H:%M:%S')

In [4]:
from scipy.stats import norm
# Load the dataset
file_path = 'BS_Model_Dataset.csv'
data = pd.read_csv(file_path)



/var/folders/bn/0_n9dq9s6m51l0sffgs_yvtw0000gn/T/ipykernel_21004/329590916.py:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [5]:
# Parameter initialization
mu = 0.05  # Drift rate
sigma = data['Volatility_sigma'].mean()  # Average volatility
lambda_jump = 0.1  # Jump intensity (annualized jump frequency)
jump_mu = -0.02  # Mean of the jump size (log-normal)
jump_sigma = 0.1  # Standard deviation of the jump size
risk_free_rate = data['Risk_Free_Rate_r'].iloc[0]  # Risk-free rate
time_step = 1 / 252  # Daily time step (assuming 252 trading days per year)
num_paths = 1000  # Number of simulation paths

In [6]:
def simulate_jump_diffusion(S0, T, mu, sigma, lambda_jump, jump_mu, jump_sigma, steps):
    dt = T / steps
    prices = np.zeros((steps + 1, num_paths))
    prices[0] = S0
    
    for t in range(1, steps + 1):
        # Brownian motion component
        dW = np.random.normal(0, np.sqrt(dt), num_paths)
        # Jump process
        jumps = np.random.poisson(lambda_jump * dt, num_paths)
        jump_size = np.random.normal(jump_mu, jump_sigma, num_paths)
        # Update prices
        prices[t] = prices[t - 1] * np.exp(
            (mu - 0.5 * sigma**2) * dt + sigma * dW + jumps * jump_size
        )
    
    return prices

# Simulate price paths
S0 = data['Stock_Price_S'].iloc[0]
T = data['Time_to_Maturity_T'].iloc[0]
steps = int(T / time_step)
prices = simulate_jump_diffusion(S0, T, mu, sigma, lambda_jump, jump_mu, jump_sigma, steps)


In [7]:
def calculate_greeks(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    return delta, gamma


In [8]:
prices = df['Price']
Strike_Price_K = df['Strike_Price_K'] 
Time_to_Maturity_T = df['Time_to_Maturity_T']
Risk_Free_Rate_r = df['Risk_Free_Rate_r'] 
Volatility_sigma = df['Volatility_sigma']

In [ ]:
# Dynamic hedging with risk management
def dynamic_hedging_with_risk_management(prices, K, T, r, sigma, max_delta_exposure, max_gamma_exposure):
    portfolio_value = 0
    hedging_cost = 0
    delta_prev = 0
    gamma_total = 0  # Cumulative Gamma exposure
    
    for t in range(prices.shape[0]):
        S_t = prices[t].mean()  # Ensure we are working with a single scalar price (e.g., mean of the paths)
        delta, gamma = calculate_greeks(S_t, K, T - t * time_step, r, sigma)
        
        # Ensure delta and gamma are scalars
        delta = np.mean(delta) if isinstance(delta, np.ndarray) else delta
        gamma = np.mean(gamma) if isinstance(gamma, np.ndarray) or isinstance(gamma, pd.Series) else gamma
        
        # Check Delta and Gamma exposures
        delta_change = abs(delta - delta_prev)  # Delta change must be scalar
        if delta_change > max_delta_exposure:  # Scalar comparison
            print(f"Time {t}: Delta adjustment exceeds limit, capping adjustment.")
            delta = delta_prev + np.sign(delta - delta_prev) * max_delta_exposure
        
        # print(f'gamma value: {gamma} gamma type: {type(gamma)} max_gamma_exposure: {max_gamma_exposure} type: {type(max_gamma_exposure)}')
        if abs(gamma) > max_gamma_exposure:  # Scalar comparison for Gamma
            print(f"Time {t}: Gamma exposure exceeds limit, capping Gamma adjustment.")
            gamma = np.sign(gamma) * max_gamma_exposure
        
        # Adjust hedging positions
        hedging_cost += delta_change * S_t
        portfolio_value += (delta - delta_prev) * S_t
        delta_prev = delta
        gamma_total += gamma  # Track cumulative Gamma exposure
    
    return portfolio_value, hedging_cost, gamma_total

# Execute the hedging strategy with risk limits
max_delta_exposure = 0.1  # Maximum allowed change in Delta
max_gamma_exposure = 0.5  # Maximum allowed Gamma exposure

portfolio_value, hedging_cost, gamma_total = dynamic_hedging_with_risk_management(
    prices, Strike_Price_K, Time_to_Maturity_T, Risk_Free_Rate_r, Volatility_sigma, max_delta_exposure, max_gamma_exposure
)

print("Final portfolio value:", portfolio_value)
print("Hedging cost:", hedging_cost)
print("Total Gamma exposure:", gamma_total)

gamma value: 0.22056319134665456 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.223777528359963 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.23652116449938673 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.21869778097379614 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.22988925724751344 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.21721476673426512 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.2205222135549657 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.21761805984951052 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.23367463776650182 gamma type: <class 'numpy.float64'> max_gamma_expo

/Users/jacksparrow/NYU_schoolwork/2024/venv/lib/python3.13/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma value: 0.0 gamma type: <class 'numpy.float64'> max_gamma_exposure: 0.5 type: <class 'float'>
gamma valu

In [ ]:

# Dynamic hedging process
def dynamic_hedging(prices, K, T, r, sigma):
    portfolio_value = 0
    hedging_cost = 0
    delta_prev = 0
    
    for t in range(prices.shape[0]):
        S_t = prices[t]
        delta, gamma = calculate_greeks(S_t, K, T - t * time_step, r, sigma)
        
        # Adjust hedging positions
        hedging_cost += np.abs(delta - delta_prev) * S_t
        portfolio_value += (delta - delta_prev) * S_t
        delta_prev = delta
    
    return portfolio_value, hedging_cost

# Execute the hedging strategy
K = data['Strike_Price_K'].iloc[0]
portfolio_value, hedging_cost = dynamic_hedging(prices, K, T, risk_free_rate, sigma)
print("Final portfolio value:", portfolio_value)
print("Hedging cost:", hedging_cost)
